# Tarea 2

Ignacio Carter y Matías Garrido

- Usando las funciones anteriores, o desarrollando las propias, escriba una función que pueda valorizar, por simulación de Montecarlo, calls y puts sobre bonos cero cupón con el modelo de HW. Considere puts y calls de plazo máximo igual a 1Y.
- Su función debe poder recibir la semilla.
- Compare sus resultados con las fórmulas del modelo y muestre que diferencias obtiene usando 1000 simulaciones y una semilla predeterminada.
- Valorice también por simulación de Montecarlo el bono cupón cero a 1Y. Muestre que diferencias obtiene respecto al valor de mercado del bono usando 1000 simulaciones y una semilla predeterminada.

## Valorización de opciones con modelo HW

In [1]:
from finrisk import QC_Financial_3 as Qcf
from typing import List, Tuple, Callable
from scipy.interpolate import interp1d

import sys
sys.path.insert(1, '../modules')
# from modules
import auxiliary as aux

from numpy import random as rnd
from scipy.stats import norm
import plotly.express as px
import pandas as pd
import numpy as np
import math

Importar la curva cero cupon

In [2]:
curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [3]:
frmt = {'tasa': '{:.4%}', 'df': '{:.6%}'}

In [4]:
curva['t'] = curva['plazo'] / 365.0

In [5]:
curva['rate'] = np.log(1 / curva['df'])/( curva['plazo'] / 365.0)

In [6]:
curva.head().style.format(frmt)

,plazo,tasa,df,t,rate
0,1,0.0811%,99.999778%,0.002740,0.000811
1,7,0.0841%,99.998388%,0.019178,0.000841
2,14,0.0780%,99.997010%,0.038356,0.000780
3,21,0.0774%,99.995549%,0.057534,0.000774
4,33,0.0781%,99.992942%,0.090411,0.000781


In [7]:
zcurva = interp1d(curva['t'],
                  curva['rate'],
                  kind='cubic',
                  fill_value="extrapolate")

In [8]:
def zrate(t: float) -> float:
    return zcurva(t)


def dzrate(t: float) -> float:
    delta = .0001
    return (zcurva(t + delta) - zrate(t)) / delta


def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

In [9]:
def zero_hw(r: float, gamma: float, sigma: float,
            zrate: float, fwd: float, t: float, T: float) -> float:
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

In [10]:
def a_hw(zrate: float, fwd, gamma: float, sigma: float, t: float, T: float,
         verbose = False):
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

In [11]:
def b_hw(gamma: float, t: float, T: float) -> float:
    """
    Calcula el valor de la función B(t,T) que interviene en la fórmula
    para el valor de un bono cupón cero en el modelo de HW.
    
    params:
    
    - gamma: intensidad de reversión del modelo HW
    - t:
    - T:
    
    return:
    
    - valor de la función B(t, T)
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

In [12]:
def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

In [13]:
def theta_fwd_measure(t: float, T:float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux - b_hw(gamma, t, T)*sigma**2

In [14]:
def sims_hw_many(gamma, sigma, theta_fwd_measure, T, r0, num_sim, num_steps, seed = None):
    """
    Simulaciones de la tasa por el modelo de Hull White
    """
    dt = 1 / 264.0
    num_steps += 1
    
    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    np.random.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = np.random.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    for i in range(1, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta_fwd_measure(i * dt, T)
    
    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
    return sim

In [15]:
def option_payoff(z: float, strike,e):
    """
    Payoff de la opcion
    """
    payoff = max(e*z - e*strike,0)
    return payoff

In [16]:
def option_price(strike,r,gamma,sigma,zrate,fwd,t,Tb,To,e):
    """
    Función calcula el precio de una call o put sobre bonos cupón cero con el modelo de HW. Se tomará como fecha
    de vencimiento 1Y.
    """
    z_tb = zero_hw(r, gamma, sigma, zrate, fwd, t, Tb)
    z_to = zero_hw(r, gamma, sigma, zrate, fwd, t, To)
    d1 = get_d1(z_tb,z_to,strike,gamma,Tb,To)
    d2 = get_d2(z_tb,z_to,strike,gamma,Tb,To)
    
    price = z_to*(e*(z_tb/z_to)*norm.cdf(e*d1) - e*strike*norm.cdf(e*d2))
        
    return price

In [17]:
def s_z(gamma:float,
        sigma:float,
        Tb: float,To: float) -> float:
    
    """
    Calcula la funcion Sz
    """
    
    B = b_hw(gamma,To,Tb)
    aux1 = 1 - math.exp(-2*gamma*To)
    aux2 = (sigma**2)/(2*gamma)
    sz = B*math.sqrt(aux2*aux1)
    return sz

In [18]:
def get_d1(z_tb,z_to,strike,gamma,Tb,To):
    
    """
    Calcula la funcion d1 de formula de opciones
    """

    sz = s_z(gamma,sigma,Tb,To)
    aux1 = math.log(z_tb/(strike*z_to))
    d1 = (aux1 + 0.5*sz**2)/sz
    return d1

In [19]:
def get_d2(z_tb,z_to,strike,gamma,Tb,To):
    
    """
    Calcula la funcion d2 de formula de opciones
    """
    
    d1 = get_d1(z_tb,z_to,strike,gamma,Tb,To)
    sz = s_z(gamma,sigma,Tb,To)
    d2 = d1 - sz
    return d2

Parámetros iniciales

In [20]:
#Parametros iniciales
num_sim = 10000
num_steps = 264
seed = 1234
sigma = .015
gamma = .5
r0 = zrate(0)
t = 0
To = 1
Tb = 2
strike = 1

Simluaciones de la tasa r por método de Monte Carlos y cálculo de bono cero cupón

In [21]:
#Simulaciones de MC de r
z = []
T = 1
r = sims_hw_many(gamma, sigma, theta_fwd_measure,To, r0, num_sim, num_steps, seed)

In [22]:
# r_c toma la tasa que simula hasta el año (264 steps)
# s va a guardar las N simulaciones para posteriormente calcular el df con 263 steps
# z almacena el valor de bono cero cupón a 1Y

s = []
for i in range(num_sim):
    r_c = r[i][-1]
    s.append(r[i][:-1])
    z.append(zero_hw(r_c, gamma, sigma, zrate, fwd, To, Tb))

Calcular factor de descuento df

In [23]:
df = 0
dt = 1/365

#Calcular factor de descuento (df)
for sim in s:
    df += math.exp(-dt * np.sum(sim))
df = df/num_sim
print("Factor de descuento",df)

Factor de descuento 0.9995374408620068


Calcular el precio de las opción por método de Monte Carlos

In [24]:
#Calcular Precio por simulacion de MC
payoff_call = []
payoff_put = []

for i in range(len(z)):
    payoff_call.append(df*option_payoff(z[i],strike,1))
    payoff_put.append(df*option_payoff(z[i],strike,-1))
    
call_mc = np.average(payoff_call)    
put_mc = np.average(payoff_put)
print("Tiempo vencimiento bono es",Tb,"y tiempo vencimiento de la opcion es", To)
print("Precio opcion call MC:",call_mc)
print("Precio opcion put MC:",put_mc)

Tiempo vencimiento bono es 2 y tiempo vencimiento de la opcion es 1
Precio opcion call MC: 0.0035041210810816863
Precio opcion put MC: 0.0038864181401477333


## Cálculo del precio de la opción con fórmula analítica

In [25]:
price_call = option_price(strike,r0,gamma,sigma,zrate,fwd,t,Tb,To,1)

print("Tiempo vencimiento bono es",Tb,"y tiempo vencimiento de la opcion es", To)
print("Precio Call: ",price_call)
price_put = option_price(strike,r0,gamma,sigma,zrate,fwd,t,Tb,To,-1)
print("Precio Put: ",price_put)

Tiempo vencimiento bono es 2 y tiempo vencimiento de la opcion es 1
Precio Call:  0.0035283883401179164
Precio Put:  0.003960769916741911


Se obtiene un precio de las opciones Call y Put practicamente iguales tanto con el método de la simulacion
de Monte Carlo como la fórmula analítica realizando 10000 simulaciones, ya que con 1000 simulaciones la precision es muy baja. El error de precisión se puede deber al número de simulaciones. 
Si el número de simulaciones aumentara, la precisión del cálculo de la opción seria aún mas precisa.

### Tests

In [29]:
import hull_white as hw

call = hw.zcb_call_put(
    hw.CallPut.CALL,
    strike=strike,
    to=To,
    tb=Tb,
    r0=r0,
    zo=math.exp(-zrate(To) * To),
    zb=math.exp(-zrate(Tb) * Tb),
    gamma=gamma,
    sigma=sigma)

put = hw.zcb_call_put(
    hw.CallPut.PUT,
    strike=strike,
    to=To,
    tb=Tb,
    r0=r0,
    zo=math.exp(-zrate(To) * To),
    zb=math.exp(-zrate(Tb) * Tb),
    gamma=gamma,
    sigma=sigma)

print(call)
print(put)

0.0035283883401179272
0.00396076991674188


## Valorización bono cero cupón a 1Y

In [26]:
df_sim = 0

# Calcular factor de descuento (df) con las simulaciones
for sim in s:
    df_sim += math.exp(-dt * np.sum(sim))
df_sim = df_sim/num_sim

# Factor de descuento a 1Y
df_mkt = curva['df'].iloc[15]

bono_sim = df_sim*np.average(z)
bono_mkt = df_mkt

print("Precio Bono Cero Cupón con simulaciones de MC", bono_sim)
print("Precio Bono Cero Cupón de mercado", bono_mkt)

Precio Bono Cero Cupón con simulaciones de MC 0.9991551438029409
Precio Bono Cero Cupón de mercado 0.9992978688293875


La diferencia entre el precio del bono cero cupón de mercado y simulado es pequeña. Tiene un pequeño error de precisión. 

**AD:** ¿Pequeño error de simulación? Miren la diferencia en tasa. Al comentario le falta análisis.

In [27]:
tasa_sim = -.5*math.log(bono_sim)
print(f'{tasa_sim:.6%}')

tasa_mkt = -.5*math.log(bono_mkt)
print(f'{tasa_mkt:.6%}')

0.042261%
0.035119%
